In [ ]:
from preprocessing.dataset import dataset
from torch.utils.data import DataLoader
from training.trainer import trainer
from models.vae import VAE
from models.ode import ode
from evaluation.evaluator import evaluator
import torch.optim as optim
from sklearn.model_selection import train_test_split
import json
import torch
import os
import sys
from copy import copy
from torchsummary import summary
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
data = dataset(data_file='QCD_LLP_samples/root-files/qcd_100k.root',
                    sg_files=['QCD_LLP_samples/h5-files/500GeV_n3_events_100k_1mm_pileup.h5', 'QCD_LLP_samples/h5-files/100GeV_n3_events_100k_1mm_pileup.h5'],
                    bkg_files=['QCD_LLP_samples/h5-files/QCD_multijet_events_200k_pileup.h5'])
  
with open('code/utils/hyperparameters.json') as f:
        hyperparameters = json.load(f)

In [ ]:
BATCH_SIZE = hyperparameters['BATCH_SIZE']
lr = hyperparameters['LEARNING_RATE']
num_epochs = hyperparameters['EPOCHS']
latent_dim = hyperparameters['LATENT_DIM']
    
train_loader = DataLoader(torch.tensor(data.data_train, dtype=torch.float32), batch_size=BATCH_SIZE, shuffle=True, num_workers=10)
valid_loader = DataLoader(torch.tensor(data.data_valid,dtype=torch.float32), batch_size=BATCH_SIZE, shuffle=True, num_workers=10)

In [ ]:
bdt_model = DecisionTreeClassifier(random_state=0)
bdt_model.fit(data.data_train, data.labels_train)

In [ ]:
bdt_evaluator = evaluator(model=bdt_model, test_data=data.data_test, labels=data.labels_test)
bdt_evaluator.roc_curve()
bdt_evaluator.confusion_matrix()
bdt_evaluator.classification_report()